In [1]:
import pandas as pd
import sqlite3
print("libarires imported sucessfully")


libarires imported sucessfully


In [2]:
orders = pd.read_csv("/content/sample_data/orders.csv")

print("Orders CSV loaded")
orders.head()

Orders CSV loaded


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("/content/sample_data/users.json")


print("Users JSON loaded")
users.head()

Users JSON loaded


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect(':memory:')

with open("/content/sample_data/restaurants.sql", 'r') as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants table loaded")
restaurants.head()

Restaurants table loaded


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")

print("Final dataset created")
df.head()

Final dataset created


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
df = df.drop(columns=['restaurant_name_x'])
df = df.rename(columns={'restaurant_name_y': 'restaurant_name'})
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')
print("Data cleaned successfully")
df.head()

Data cleaned successfully


,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
0,1,2508,450,2023-02-18,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
df[df['membership'] == 'Gold'] \
.groupby('city')['total_amount'] \
.sum() \
.sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [8]:
df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [9]:
(df.groupby('user_id')['total_amount'].sum() > 1000).sum()

np.int64(2544)

In [33]:
df.groupby(
    pd.cut(df['rating'], [3, 3.5, 4, 4.5, 5]),
    observed=True
)['total_amount'].sum().sort_values(ascending=False)


,total_amount
rating,
"(4.5, 5.0]",2197030.75
"(4.0, 4.5]",1960326.26
"(3.0, 3.5]",1881754.57
"(3.5, 4.0]",1717494.41


In [11]:
df[df['membership'] == 'Gold'] \
.groupby('city')['total_amount'].mean() \
.sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [12]:
df.groupby('cuisine').agg(
    restaurants=('restaurant_id','nunique'),
    revenue=('total_amount','sum')
).sort_values(['restaurants','revenue'], ascending=[True,False])

,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [13]:
gold_pct = (df[df['membership']=='Gold'].shape[0] / df.shape[0]) * 100
round(gold_pct, 2)


49.87

In [14]:
df.groupby('restaurant_name').agg(
    orders=('order_id','count'),
    aov=('total_amount','mean')
).query('orders < 20').sort_values('aov', ascending=False)

,orders,aov
restaurant_name,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [15]:
df.groupby(['membership','cuisine'])['total_amount'] \
.sum().sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [16]:
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [17]:
gold_orders = df[df['membership']=='Gold'].shape[0]
gold_orders

4987

In [18]:
hyderabad_revenue = round(df[df['city']=='Hyderabad']['total_amount'].sum())
hyderabad_revenue


1889367

In [19]:
distinct_users = df['user_id'].nunique()
distinct_users

2883

In [20]:
gold_aov = round(df[df['membership']=='Gold']['total_amount'].mean(),2)
gold_aov


np.float64(797.15)

In [21]:
high_rating_orders = df[df['rating']>=4.5].shape[0]
high_rating_orders


3374

In [22]:
top_gold_city = df[df['membership']=='Gold'].groupby('city')['total_amount'].sum().idxmax()
orders_top_gold_city = df[(df['membership']=='Gold') & (df['city']==top_gold_city)].shape[0]
orders_top_gold_city


1337

In [23]:
df.shape[0]

10000

In [24]:
df.groupby('restaurant_name').agg(
    orders=('order_id','count'),
    aov=('total_amount','mean')
).query('orders < 20').sort_values('aov', ascending=False)

,orders,aov
restaurant_name,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [25]:
orders.columns, users.columns


(Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
        'restaurant_name'],
       dtype='object'),
 Index(['user_id', 'name', 'city', 'membership'], dtype='object'))

In [26]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [27]:
df.shape[0]


10000

In [28]:
df.isnull().sum()


,0
order_id,0
user_id,0
restaurant_id,0
order_date,0
total_amount,0
name,0
city,0
membership,0
restaurant_name,0
cuisine,0


In [31]:
df[df['user_id'] == df['user_id'].iloc[0]]


,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating,quarter
0,1,2508,450,2023-02-18,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,2023Q1
7724,7725,2508,99,2023-05-04,207.32,User_2508,Hyderabad,Regular,Restaurant_99,Italian,4.0,2023Q2
9323,9324,2508,39,2023-11-10,998.38,User_2508,Hyderabad,Regular,Restaurant_39,Indian,4.8,2023Q4
